In [2]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy
import os 
import re
import json
import pickle

In [2]:
# vocab_path = os.getcwd()+'/bert_model/vocab_low_data.txt'
# model_path = os.getcwd()+'/blabla/'

In [4]:
# tokenizers = BertTokenizer.from_pretrained(vocab_path)
# model = BertForMaskedLM.from_pretrained(pretrained_model_name_or_path=model_path, return_dict = True, output_hidden_states = True)

In [5]:
model = AutoModelForMaskedLM.from_pretrained("Shushant/nepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

In [6]:
tokenizers = AutoTokenizer.from_pretrained("Shushant/nepaliBERT")

In [7]:
# pickle.dump(model,open('bert_model','wb'))
# pickle.dump(tokenizers,open('tokenizers','wb'))


In [3]:
# model = pickle.load(open('bert_model','rb'))


In [4]:
# tokenizers = pickle.load(open('tokenizers','rb'))

In [5]:
tokenizers.tokenize("के मौजुदा लोकतान्त्रिक व्यवस्था राज्य पुनःसंरचनासँग जोडिएका हिजोका सवालहरूलाई यथास्थितिमा छोडेर सबल होला?")

['क',
 'मौ',
 '##ज',
 '##दा',
 'लोक',
 '##तान',
 '##तर',
 '##िक',
 'वय',
 '##वस',
 '##था',
 'राज',
 '##य',
 'पन',
 '##ः',
 '##सर',
 '##चना',
 '##सग',
 'जोडिएका',
 'हिजोका',
 'सवाल',
 '##हर',
 '##लाई',
 'यथा',
 '##स',
 '##थिति',
 '##मा',
 'छोड',
 '##र',
 'सबल',
 'होला',
 '?']

In [11]:
# text = 'अनि तेस्रो चिन्ता मौसम परिवर्तनले हिमशिखरहरूमा परेका आघातसँगसँगै सिमानावारिपारि नदीले ल्याएका प्रकोपहरू कसरी सम्हाल्ने'
# marked_text = " [CLS] "+text+" [SEP] "
# tokenized_text = tokenizer.tokenize(marked_text)
# indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# segments_ids = [1] * len(indexed_tokens)

# tokens_tensors = torch.tensor([indexed_tokens])
# segments_tensors = torch.tensor([segments_ids])

In [53]:
# with torch.no_grad():
#     outputs = model(tokens_tensors, segments_tensors)
#     hidden_states = outputs.hidden_states
# #     print(hidden_states[-1])
#     token_embeddings = hidden_states[-1]
    
#     token_embeddings = torch.squeeze(token_embeddings, dim = 0)
    
#     list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
#     print(list_token_embeddings)

In [13]:
texts = ['तर','दुधमा तर बसेन|','तिम्रो घर आउन मन लाग्छ तर अल्छि लाग्छ']

In [14]:
def bert_text_preparation(text, tokenizer ):
    """Preparing input for BERT"""
    
    marked_text = " [CLS] " + text + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    return tokenized_text, tokens_tensors, segments_tensors

In [15]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    # Gradient claculation id disabled 
    # Model is in inference mode
    
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 
        hidden_states = outputs.hidden_states
    
    # Getting embeddings from final Bert Layer
    tokens_embeddings = hidden_states[-1]
    # Collasping the tensor into 1-dimension 
    tokens_embeddings = torch.squeeze(tokens_embeddings, dim = 0)
    # Converting torchtensors to lists 
    list_token_embeddings = [token_embed.tolist() for token_embed in tokens_embeddings]
    
    return list_token_embeddings 

In [16]:
# tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizer)
# target_word_embeddings = []

# for text in texts:
#     tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizer)
#     list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     ## list_token_embeddings has embeddings of the given words
# #     word_index = tokenized_text.index('तर')
#     word_embeddings = [list_token_embeddings[token] for token in tokenized_text]
# #     word_embedding = list_token_embeddings[word_index]
# #     print(word_embedding)
# #     target_word_embeddings.append(word_embedding)

In [17]:
# tokenized_text

In [18]:
# type(tokenized_text[0])

In [19]:
target_word_embeddings = []

for text in texts:
    tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizers)
    list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     print(len(list_token_embeddings))
    ## list_token_embeddings has embeddings of the given words
    word_index = tokenized_text.index('तर')
    word_embedding = list_token_embeddings[word_index]
#     print(word_embedding)
    target_word_embeddings.append(word_embedding)

In [20]:
target_word_embeddings[0] == target_word_embeddings[1]

False

In [21]:
len(target_word_embeddings)

3

In [22]:
# list_of_distances = []
# for text1, embed1 in zip(texts, target_word_embeddings):
#     for text2, embed2 in zip(texts, target_word_embeddings):
#         cos_dist = 1 - cosine(embed1,embed2)
#         list_of_distances.append([text1, text2, cos_dist])


# distances_df = pd.DataFrame(list_of_distances, columns = ['text1','text2','distance'])


In [6]:
# df = pd.read_csv("finalData.csv")
df = pd.read_csv('collected_labeled_data.csv')

In [13]:
# df.drop(df.columns[-1],axis = 1, inplace = True)

In [14]:
df['label'].unique()

array([2, 1, 0], dtype=int64)

In [15]:
# df.to_csv('collected_labeled_data.csv',index = False)

In [51]:
# train_X, test_X = train_test_split(df, test_size = 0.2)

In [52]:
# train_X.to_csv('train.csv',index = False)
# test_X.to_csv('test.csv',index = False)

In [53]:
def check_len(text):
    txt = text.split(' ')[:20]
    return ' '.join(txt)

In [52]:
# df['text'] = df['text'].apply(check_len)

In [55]:
# def get_word_embeddings(text):
#     tokenizer = tokenizers
#     tokenized_text, tokens_tensors, segments_tensors = bert_text_preparation(text, tokenizer)
#     list_token_embeddings = get_bert_embeddings(tokens_tensors, segments_tensors, model)
#     ## list_token_embeddings has embeddings of the given words
#     return list_token_embeddings

In [57]:
stopwords= stopwords.words("nepali")

In [46]:
words = ['अक्सर','आदि','कसरी','अन्तर्गत','अर्थात','अर्थात्','अलग','आयो','उदाहरण','एकदम','राम्रो','बिरुद्ध','बिशेष']

In [47]:
stopwords = list(set(stopwords).difference(set(words)))

In [48]:
def clean_text(text):
    text = text.split(' ')
    clean_text_list = []
    for word in text:
        if word not in stopwords:
            clean_text_list.append(word)
    clean_text = ' '.join(clean_text_list)
    txt = re.sub(r"[|a-zA-z.'#0-9@,:?'\u200b\u200c\u200d!/&~-]",'',clean_text)
    
    return txt

In [49]:
clean_text("घाम जति लग्यो हामीलाई तेती राम्रो हुन्छ ")

'घाम जति लग्यो हामीलाई तेती राम्रो '

In [15]:
# df['text'] = df['text'].apply(clean_text)

In [16]:
# distances_df[distances_df.text1 == 'तर']

In [58]:
df.head()

,text,label
0,बजार ले जसरी ट्रेन्ड चेन्ज गर्यो यो हेर्दा तत्...,2
1,1000 अंकले घटेको नेप्से 200 अंकले बढ्नु ठूलो क...,1
2,होइन यो सानिमा बैंक ले bonus घोसणा गरेको २ महि...,2
3,"खैँ MBJC प्रति कित्तामा रू,10/-ले बढेर आज रू,1...",2
4,राम्रो भयो️️,1


In [7]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]


    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
    token_vecs = hidden_states[-2][0]

    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [8]:
get_bert_embedding_sentence("नेपाल को ससकृती ध्वस्त पार्ने योजना")

array([ 2.91083171e-03,  2.76484221e-01,  4.30840701e-02,  2.34462023e-01,
        1.93695650e-01,  8.08570236e-02,  1.67143255e-01, -1.21491820e-01,
        1.98826805e-01, -2.85235792e-01,  8.43919590e-02,  4.07811016e-01,
       -9.35231894e-02,  2.51754314e-01, -2.34685354e-02,  2.99455728e-02,
        1.88694015e-01, -3.43165666e-01, -1.53445616e-01,  3.77470464e-01,
        1.72779784e-01, -1.76885203e-01,  1.78813785e-01, -4.30221826e-01,
        9.03602466e-02, -1.43045276e-01,  3.37989405e-02,  2.06438884e-01,
        6.37542754e-02,  2.21104681e-01,  1.67193517e-01,  4.74679053e-01,
        3.89956608e-02, -5.72416842e-01, -2.64643490e-01, -5.66417336e-01,
        1.11003555e-01,  4.15074408e-01, -2.21883088e-01,  8.65613967e-02,
        1.92128755e-02,  2.18923986e-01,  4.32191789e-01, -9.03057232e-02,
        7.46855587e-02,  3.52693230e-01, -4.17815506e-01,  3.04982007e-01,
       -2.55210251e-01,  1.57838210e-01, -3.86268824e-01, -1.11495960e+00,
        3.70744109e-01, -

In [18]:
df=df.drop(df[df['label']==2].index)

In [19]:
df.dropna(inplace = True)

In [20]:
df['word_embeddings'] = df['text'].apply(get_bert_embedding_sentence)

In [21]:
df.head()

,text,label,word_embeddings
1,1000 अंकले घटेको नेप्से 200 अंकले बढ्नु ठूलो क...,1,"[-0.261957, 0.5387599, -0.16427921, 0.2529497,..."
4,राम्रो भयो️️,1,"[-0.30638534, 0.95327806, -0.38146958, 0.54328..."
6,जानकारीको लागि धन्यवाद रामहरी ब्रदर,1,"[0.317741, 0.65826416, -0.12128323, 0.06808527..."
18,"भारत-मधेस र नेपाल-चीन सम्बन्ध विग्रन्छ, मधेसी ...",0,"[0.15587386, 0.6471652, -0.13978036, -0.099823..."
25,"लेखनाथ न्यौपानेको खुलासा,महाधिबेशनमा एमसीसीको ...",0,"[-0.140361, 0.88074034, 0.05382303, 0.11658585..."


In [14]:
# df.to_csv('embedding_data.csv',index = False)

In [22]:
X,y = df['word_embeddings'], df['label']

In [23]:
# scaler = StandardScaler()
# pca = PCA(n_components = 768)

In [24]:
# scaled_X = scaler.fit_transform(X.tolist())
# pca_X = pca.fit_transform(scaled_X)

In [25]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [26]:
svc = SVC()

In [27]:
# train_X = [i[0] for i in train_X]
# test_X = [i[0] for i in test_X]

In [28]:
# train_X[0][0].shape

In [29]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC()

In [30]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [31]:
print(confusion_matrix(test_y, svc_pred))

[[406  70]
 [ 84 652]]


In [32]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       476
           1       0.90      0.89      0.89       736

    accuracy                           0.87      1212
   macro avg       0.87      0.87      0.87      1212
weighted avg       0.87      0.87      0.87      1212



In [33]:
accuracy_score(test_y, svc_pred)

0.8729372937293729

In [40]:
f1_score(test_y, svc_pred)

0.8943758573388204

In [53]:
svc.predict(np.array(get_bert_embedding_sentence("नराम्रो कुरा नगरेकै बेश").tolist()).reshape(1,-1))

array([0], dtype=int64)

In [54]:
svc.predict(np.array(get_bert_embedding_sentence("मलाई पढ्न मनपर्छ").tolist()).reshape(1,-1))

array([1], dtype=int64)

In [46]:
df.head()

,text,label,word_embeddings
1,1000 अंकले घटेको नेप्से 200 अंकले बढ्नु ठूलो क...,1,"[-0.261957, 0.5387599, -0.16427921, 0.2529497,..."
4,राम्रो भयो️️,1,"[-0.30638534, 0.95327806, -0.38146958, 0.54328..."
6,जानकारीको लागि धन्यवाद रामहरी ब्रदर,1,"[0.317741, 0.65826416, -0.12128323, 0.06808527..."
18,"भारत-मधेस र नेपाल-चीन सम्बन्ध विग्रन्छ, मधेसी ...",0,"[0.15587386, 0.6471652, -0.13978036, -0.099823..."
25,"लेखनाथ न्यौपानेको खुलासा,महाधिबेशनमा एमसीसीको ...",0,"[-0.140361, 0.88074034, 0.05382303, 0.11658585..."


In [42]:
# import pickle 
# pickle.dump(svc,open('svc_sentiment','wb'))

In [43]:
svc_sentiment = pickle.load(open('svc_sentiment','rb'))

In [45]:
# svc_sentiment.predict(test_X.tolist())

array([1, 1, 0, ..., 0, 1, 0])

In [49]:
nb = GaussianNB()

In [50]:
nb.fit(train_X.tolist(), train_y)
# nb.fit(train_X, train_y)

GaussianNB()

In [51]:
nb_pred = nb.predict(test_X.tolist())
# nb_pred = nb.predict(test_X)

In [52]:
print(confusion_matrix(test_y, nb_pred))

[[518 111]
 [277 303]]


In [53]:
print(classification_report(test_y, nb_pred))

              precision    recall  f1-score   support

           0       0.65      0.82      0.73       629
           1       0.73      0.52      0.61       580

    accuracy                           0.68      1209
   macro avg       0.69      0.67      0.67      1209
weighted avg       0.69      0.68      0.67      1209



In [54]:
accuracy_score(test_y, nb_pred)

0.6790736145574855

In [35]:
svc.predict(np.array(get_bert_embedding_sentence("नराम्रो").tolist()).reshape(1,-1))

array([1], dtype=int64)

In [39]:
svc.predict(np.array(get_bert_embedding_sentence("कति राम्री केटि हो").tolist()).reshape(1,-1))

array([1], dtype=int64)

In [12]:
rfc = RandomForestClassifier()

In [13]:
n_estimators = [100,200,300,400]
max_features = ['auto','sqrt']
max_depth = [5,10,20,40,50]
min_samples_split = [2,5,10,20]
min_samples_leaf = [1,2,4,5]
bootstrap = [True, False]

In [14]:
random_grid = {
    'n_estimators':n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}

In [15]:
cv_rfc = GridSearchCV(estimator = rfc, param_grid = random_grid, cv =5)

In [16]:
cv_rfc

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [5, 10, 20, 40, 50],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2, 4, 5],
                         'min_samples_split': [2, 5, 10, 20],
                         'n_estimators': [100, 200, 300, 400]})

In [17]:
cv_rfc.fit(X.tolist(),y)

In [37]:
cv_rfc.best_params_

{'bootstrap': False,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 100}

In [67]:
rfc_new = RandomForestClassifier(bootstrap =False, max_depth = 10, max_features = 'auto', min_samples_leaf = 1, min_samples_split = 10, n_estimators = 1200)

In [68]:
rfc_new.fit(train_X.tolist(),train_y)
# rfc.fit(train_X,train_y)

RandomForestClassifier(bootstrap=False, max_depth=10, min_samples_split=10,
                       n_estimators=1200)

In [69]:
rfc_predict = rfc_new.predict(test_X.tolist())
# rfc_predict = rfc.predict(test_X)

In [70]:
print(classification_report(test_y, rfc_predict))

              precision    recall  f1-score   support

           0       0.86      0.82      0.84       476
           1       0.89      0.91      0.90       736

    accuracy                           0.88      1212
   macro avg       0.87      0.87      0.87      1212
weighted avg       0.87      0.88      0.87      1212



In [71]:
f1_score(test_y, rfc_predict)

0.898861352980576

In [62]:
gfc = GradientBoostingClassifier()

In [63]:
gfc.fit(train_X.tolist(), train_y)
# gfc.fit(train_X, train_y)

GradientBoostingClassifier()

In [64]:
gfc_predict = gfc.predict(test_X.tolist())
# gfc_predict = gfc.predict(test_X)

In [65]:
print(classification_report(test_y, gfc_predict))

              precision    recall  f1-score   support

           0       0.68      0.72      0.70       629
           1       0.68      0.64      0.66       580

    accuracy                           0.68      1209
   macro avg       0.68      0.68      0.68      1209
weighted avg       0.68      0.68      0.68      1209



In [66]:
gfc.predict(np.array(get_bert_embedding_sentence("त चोर होस्").tolist()).reshape(1,-1))

array([0])

In [38]:
rfc.predict(np.array(get_bert_embedding_sentence("त चोर होस्").tolist()).reshape(1,-1))

array([0])

In [29]:
# svc.predict(pca.transform(scaler.transform(get_bert_embedding_sentence("एस्तो सोच राख्ने मान्छे त मर्नु बरु ").reshape(1,-1))))
svc.predict(np.array(get_bert_embedding_sentence("कति नराम्रो छ्या").tolist()).reshape(1,-1))

array([0])

In [69]:
clean_text("एस्तो सोच राख्ने मान्छे त मर्नु बरु")

'एस्तो सोच राख्ने मान्छे मर्नु'

In [35]:
f1_score(test_y,svc_pred)

0.8943758573388204

In [71]:
f1_score(test_y,nb_pred)

0.6096579476861167

In [72]:
f1_score(test_y,gfc_predict)

0.6578249336870027

In [11]:
# f1_score(test_y,rfc_predict)